# Assignment 07: Raster math and sampling rasters
including working with multi-spectral imagery accessed from the cloud


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

import rasterio as rio
import rasterio.plot # Necessary to use rio.plot.show()

import geopandas as gpd
import fiona


In [ ]:
%matplotlib widget
# %matplotlib inline

In [ ]:
# Find the datasets directory in a parent directory
datasets_dir = Path('datasets')
while not datasets_dir.exists(): # so long as `datasets` is not in the path datasets_dir ...
    datasets_dir = '..' / datasets_dir # ... keep adding the parent directory to the datasets path


## Working with satellite data
One of the common, useful, land imaging satellites is the Landsat series of satellites, operated by the USGS and NASA.
The successful Landsat program is now in its 8th version, with the 9th satellite launched last month, in September 2021.
Landsat 9 will come online in several months, after testing is complete.

The Landsat 8 satellite has a number of imagers (think: cameras) that are sensitive to light in different parts of the
electromagnetic spectrum.  "Parts" in this sense are different bands of wavelengths or frequencies of light.
The figure below ([from NASA and the USGS](https://www.usgs.gov/media/images/comparison-landsat-7-and-8-bands-sentinel-2)) compares the bands of Landsat 7, Landsat 8, and Sentinel-2 (a similar satellite operated by
the European Space Agency).

<img src="dmidS2LS7Comparison.png" width="700">

We can use the fact that different surface reflect and emit light in different wavelengths to learn
about what a surface is.

We'll use these properties in this component of the assignment.

#### Reading:
For more on working with Landsat imagery, please check out https://geohackweek.github.io/raster/04-workingwithrasters/
This is an awesome tutorial, and in part what this assignment is based on.

Another resource is
https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/landsat-in-Python/.  
However, this site leans on xarray, which I have not introduced in this class (despite its awesomeness).


In [ ]:
print('Landsat on AWS:')

# LC08_L1TP_043027_20201005_20201015_01_T1
scene_path = 'http://landsat-pds.s3.amazonaws.com/c1/L8/043/027/LC08_L1TP_043027_20201005_20201015_01_T1/'
band_file = 'LC08_L1TP_043027_20201005_20201015_01_T1_B2.TIF'

with rasterio.open(scene_path + band_file) as src:
    print(src.profile)

<div class="alert alert-block alert-warning">

### Dating Landsat imagery
When is this image from?  See https://www.usgs.gov/faqs/what-naming-convention-landsat-collections-level-1-scenes?qt-news_science_products=0#qt-news_science_products
</div>

### Plot a low-resolution overview of the landsat scene

In [ ]:
# The grid of raster values can be accessed as a numpy array and plotted:
with rasterio.open(scene_path + band_file) as src:
    oviews = src.overviews(1) # list of overviews from biggest to smallest
    print('These are the decimation factors that create smaller and smaller views of the data.')
    print('"Decimation" by a factor of n means plot every _nth_ pixel.')
    print(oviews)
    print('\n')
    
    oview = oviews[-1] # let's look at the smallest thumbnail
    print('Decimation factor= {}'.format(oview))
    # NOTE this is using a 'decimated read' (http://rasterio.readthedocs.io/en/latest/topics/resampling.html)
    thumbnail = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))

print('array type: ',type(thumbnail))
print(thumbnail)

plt.imshow(thumbnail)
plt.colorbar()
plt.title('Overview - Band 4 {}'.format(thumbnail.shape))
plt.xlabel('Column #')
plt.ylabel('Row #')

In [ ]:
src.meta

In [ ]:
dx = src.transform[0] # grid resolution in Easting (x)
rast_x = src.transform[2] + dx * np.arange(src.width)
dy = src.transform[4] # grid resolution in Northing (y)
rast_y = src.transform[5] + dy * np.arange(src.height)

### Read in each of bands 2-5
We want to read in data from bands 2-5, but we don't want to write code to do that to each
individual band separately. Instead we'll do that with a for loop, reading a different band
each time we go through the loop.

We're going to start by creating an empty dictionary object, named `landsat_data`, and then
use the band numbers as "keys" to read in the actual pixel values for each band.
See this small section on dictionaries for an overview: https://docs.python.org/3/tutorial/datastructures.html#dictionaries

In [ ]:
print(band_file)
scene_name = band_file[:-7]
print(scene_name)

In [ ]:
bands = np.arange(2,6) # remember: inclusive of the first value, exclusive of the last
# raster_names = ['LC08_B{}'.format(band) for band in bands]

# scene_path + band_file

# band_root = 'LC08_L1TP_043027_20210602_20210608_01_T1'
# band_root = 'LC08_L1TP_043027_20201005_20201015_01_T1'
landsat_data = {} # 
for band in bands:
    complete_path = scene_path + scene_name + '_B{}.TIF'.format(band)
    print(complete_path)
    with rio.open(complete_path) as src:

        landsat_data[band] = src.read(1)#, out_shape=(1, int(src.height // oview), int(src.width // oview)))

        landsat_data[band] = landsat_data[band].astype('f4')
        landsat_data[band][landsat_data[band]==0] = np.nan

    plt.figure()
    plt.imshow(landsat_data[band], vmax=20000)
    plt.colorbar()
    plt.title('Overview - Band {}'.format(band))
    plt.xlabel('Column #')
    plt.ylabel('Row #')

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(10,3))
ax[0].imshow(landsat_data[4], cmap='Reds',  vmax=20000)
ax[1].imshow(landsat_data[3], cmap='Greens',vmax=20000)
ax[2].imshow(landsat_data[2], cmap='Blues', vmax=20000)

<div class="alert alert-block alert-warning">

### Saturation in RGB space
Recognize different land surfaces in this landsat scene: forest, crop land, lakes, and channeled scabland.
    
In the three red, green, and blue bands, which land surface is most reflective?  In what band is this land surface
most reflective?
</div>

Let's combine these bands to plot a true color image of the data.  See [the documentation for numpy.stack](https://numpy.org/doc/stable/reference/generated/numpy.stack.html).


In [ ]:
data_stack = np.stack((landsat_data[4], landsat_data[3], landsat_data[2]), axis=2)

In [ ]:
data_stack.shape

In [ ]:
extent = (rast_x[0], rast_x[-1], rast_y[-1], rast_y[0])

fig, ax = plt.subplots(figsize=(6,6))
# ax.pcolormesh(rast_x, rast_y, data_stack)
ax.imshow(data_stack/15000, extent=extent)
ax.set_title(scene_name + '\nNatural Color')

# # Limits for Moscow
# ax.set_xlim(494000, 504000)
# ax.set_ylim(5.17e6, 5.18e6)

### Calculate the NDVI
The [Normalized Difference Vegetation Index](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index) (NDVI) 
is a simple indicator that can be used to assess whether the
target includes healthy vegetation. This calculation uses two bands of a multispectral image dataset,
the Red and Near-Infrared (NIR) bands.

In [ ]:
ndvi = (landsat_data[5] - landsat_data[4]) / (landsat_data[5] + landsat_data[4])

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
img = ax.imshow(ndvi, extent=extent)
plt.colorbar(img)
ax.set_title(scene_name + '\nNDVI')

# Moscow's location:
# ax.set_xlim(495000, 505000)
# ax.set_ylim(5.17e6, 5.18e6)

### Forests and fields
Find Moscow and Pullman, in the south of the image above.  Find Moscow Mountain.
It appears that Moscow Mountain NDVI is higher (greener) than the surrounding fields.

Let's see if we can identify whether there is a distinct break in the NDVI,
between the fields and the forest, and whether this might potentially be useful to classify
fields and forests distinctly in this October Landsat image.

My approach is to create a polygon that straddles both field and forest regions,
use this polygon to clip out a section of the NDVI raster, and then look at a
histogram of the pixel values to see if we can identify two separate peaks in NDVI
values or if the distribution of NDVI values is more continuous and therefore NDVI cannot
be used to differentiate the two land cover types.

The first thing I've done to solve this challenge is to create a rectangle in shapely 
that straddles the forest/field interface.  

In [ ]:
from shapely.geometry import Polygon
poly_lower_left = (502000, 5.180e6) # Easting, Northing coordinates
poly_width = 2000
poly_height = 5000

sample_rect = Polygon([ poly_lower_left,
                        (poly_lower_left[0]+poly_width, poly_lower_left[1]),
                        (poly_lower_left[0]+poly_width, poly_lower_left[1]+poly_height),
                        (poly_lower_left[0], poly_lower_left[1]+poly_height) ])
corner_x, corner_y = sample_rect.exterior.coords.xy # Extract the coordinates of the polygon corners, so we can plot them below


Let's plot this rectangle on the NDVI map above, to see where it is

In [ ]:
ax.plot(corner_x, corner_y, color='C1')

To efficiently crop a raster using rasterio's mask operation and the shapely object we've 
created, we need an original rasterio DatasetReader object.
Therefore, we'll just reload the two bands we need.

In [ ]:
# Calculate the NDVI for the pixels in the shapely cutout
from rasterio.mask import mask
red_rio = rio.open(scene_path + scene_name + '_B{}.TIF'.format(4))
nir_rio = rio.open(scene_path + scene_name + '_B{}.TIF'.format(5))

nir_crop, _ = mask(nir_rio, [sample_rect], crop=True)
red_crop, _ = mask(red_rio, [sample_rect], crop=True)

# Reduce dimensionality of the array first, then recast it as a float, to prepare for division
nir_crop = np.squeeze(nir_crop).astype(float)
red_crop = np.squeeze(red_crop).astype(float)
# Eliminate zeros from the cropped arrays.
nir_crop[ nir_crop == 0 ] = np.nan
red_crop[ red_crop == 0 ] = np.nan


ndvi_crop = (nir_crop - red_crop) / (nir_crop + red_crop) 

In [ ]:
plt.figure()
plt.hist(ndvi_crop.ravel(), np.arange(0, .5, .01))
plt.xlabel('NDVI')
plt.ylabel('Number of pixels with this NDVI value')

In [ ]:
mosc_vectors = datasets_dir / 'moscow' / 'moscow_vectors' / 'Moscow.gdb'
streets = gpd.read_file(mosc_vectors, layer='Centerlines')

In [ ]:
raster_crs = red_rio.crs.to_epsg() # Get the EPSG code for the landsat scene
streets = streets.to_crs(raster_crs) # Transform the streets layer so that it has the same CRS as the raster
streets.plot()

In [ ]:
# Add the streets to the map several plots above, that is still represented by the object `ax`
streets.plot(ax=ax, color='C3')

<div class="alert alert-block alert-warning">

## Exploring NDVI values
Using both the map of NDVI values above, onto which we've just plotted Moscow's streets, 
    as well as the histogram of fields and forest NDVI values, tell me about typical NDVI values...
    (no need to do any programming to get at these values, just look at the plots, zoom in, and around)
    
What are:
+ the highest NDVI values in Moscow, and what land surface are these locations?
+ the lowest NDVI values, and what land surface are these locations?
+ the NDVI values for Moscow Mountain forest?
+ the NDVI values for crop land south of Moscow Mountain?
    
Does the crop land and the forest on Moscow Mountain in fact have distinct NDVI values?
</div>

<div class="alert alert-block alert-warning">

## Evaluating impacts of the 2021 drought on urban and agricultural vegetation
You're a horticulturalist for the State of Idaho.    
Your boss asks you how the drought from this summer differently impacted crops and urban green spaces
before and after the worst of the summer heat in north-central Idaho.
    
You decide to focus on Moscow, known for its leafy streets, verdant parks, and rich surrounding 
agricultural land, and to evaluate changes over the seasons using differences in Landsat derived NDVI.
You'll use the landsat scene we were working with above as well as another great, cloud-free image
from early June of this year:
```
scene_path = 'http://landsat-pds.s3.amazonaws.com/c1/L8/043/027/LC08_L1TP_043027_20210602_20210608_01_T1/'
band_file = 'LC08_L1TP_043027_20210602_20210608_01_T1_B2.TIF'
```

    
Quantify and plot four NDVI values: 1) June in Moscow, 2) June Moscow cropland, 3) October in Moscow, and 
    4) October Moscow cropland.  For the in-Moscow measurements, calculate the average NDVI for the entire
    city (recall that we have a city limits layer that we've worked with previously, in out datasets folder).
    For the Moscow cropland, calculate the average NDVI outside the city limits, but within a 2 km buffer of
    the city limits.

Write a few sentences describing what you see in the plot you've created.
    
Recommendation (not a requirement):
> A lot of the steps you'll need to do to produce these four NDVI values will be repeated four times.  To cut down
    on the amount of code you need to write, and make your code more readable, write a function that delivers
    you the answer you need, and then call it four times.
    

</div>